**本文作者诗雅，已同步于微信公众号【气象备忘录】**

## 一、Nuging简介

Nudging是基于经验性的四维资料同化方法，通过在预报方程中增加强迫项，使模式输出结果逼近于观测资料（Observation Nudging）或再分析资料（Analysis Nudging）。这些强迫项相对于方程中的各项量级较小，保证了在每个积分步长中各物理量之间的动力平衡。

WRF提供了两种Analysis Nudging方案，分别为Grid Nudging和Spectral Nudging。Grid Nudging 是使用格点对格点的松弛强迫项逐步逼近分析场。Spectral Nudging通过设置波数进行滤波，保留大尺度波动，滤掉小尺度的波动，从而将模拟状态不断靠近大尺度驱动状态，较好地减小大尺度误差。值得注意的是，在 WRF 模式中，两种方法同化的变量不同，Grid Nudging 中默认同化的变量包括水平风、位温和水汽混合比，Spectral Nudging 中默认同化水平风、位温和位势高度。

## 二、WRF模式中Nudging设置

Nudging方法可在namelist.input中的&fdda模块进行添加，WRF模式提供了Grid Nudging、 Spectral Nudging和Observation Nudging三种方案设置，可从em_real文件下examples.namelist、README.grid_fdda和README.obs_fdda中查看相应设置。也可从 https://www.esrl.noaa.gov/gsd/wrfportal/namelist_input_options.html 网页查看每条语句含义。

### 2.1 Grid Nudging设置


![Image Name](https://cdn.kesci.com/upload/image/rgah03703f.png?imageView2/0/w/960/h/960)


上面表格是针对Upper-air层次上进行Grid Nudigng（一些文献称为 3D Analysis Nudging），若要在Surface层次上进行Grid Nudigng（Surface Analysis Nuding），需要进行下列设置，两者既可以分开也可以合在一起使用。


![Image Name](https://cdn.kesci.com/upload/image/rgah0wakhk.png?imageView2/0/w/960/h/960)


### 2.2 Spectral Nudging设置


![Image Name](https://cdn.kesci.com/upload/image/rgah1kd8q9.png?imageView2/0/w/960/h/960)


### 2.3 Observation Nudging设置


![Image Name](https://cdn.kesci.com/upload/image/rgah2nerrt.png?imageView2/0/w/960/h/960)



## 三、OBSGRID资料前处理模块

在进行Nudging之前，需要通过OBSGRID模块将little_r格式的观测资料转化为WRF能直接读取的ASCII格式文件。OBSGRID模块的作用与WRFDA系统中资料前处理程序OBSPROC类似。OBSGRID模块以及little_r格式的详细介绍可参考WRF User’s Guide [第七章Objective Analysis (OBSGRID)](https://www2.mmm.ucar.edu/wrf/users/docs/user_guide_v4/v4.4/users_guide_chap7.html) 。

### 3.1 编译OBSGRID模块

（1）将OBSGRID.tar.gz压缩包放置在WPS文件夹下，解压缩文件

tar -zxvf OBSGRID.tar.gz

（2）解压缩后，将出现一个OBSGRID 目录，进入OBSGRID文件夹

cd OBSGRID

（3）编译

./configure

./compile

./compile可能会出错，解决：在./configure之后，在configure.oa里，找到开头为CPPFLAGS这一行，删去"-C"选项，再编译就行了。如果成功，将创建可执行文件obsgrid.exe。

### 3.2、运行OBSGRID模块

OBSGRID模块运行是在WRF模式./metgrid.exe步骤之后，运行OBSGRID模块需要输入little_r格式的观测资料和met_em.d0*的模式文件。

（1）准备观测资料。观测资料须处理为little_r格式且文件命名格式为OBS:YYYY-MM-DD_HH，并将其储存在OBSGRID文件夹下。观测资料转为little_r格式网上提供了很多程序，可参考进行处理。

（2）准备met_em.d0*模式文件。将./metgrid.exe步骤生成的met_em.d0*文件链接到OBSGRID文件夹下 。

（3）修改namelist.oa文件。主要修改 &record1的时间和时间间隔、&record2的嵌套层数（grid_id）和文件名（obs_filename改为’OBS’）。

（6）运行obsgrid.exe(./obsgrid.exe)。 在namelist.oa中每更改一次grid_id（嵌套层数），就要运行一次./obsgrid.exe。

（7）运行完./obsgrid.exe之后，会出现metoa_em.d0*、OBS_DOMIANxxx、wrfsfdda_d0*。

OBS_DOMAIN1xx和OBS_DOMAIN2xx分别存放d01和d02区域内单时次的观测数据，需要借助run_cat_obs_files.csh文件将每个区域多个单时次文件合并一个文件（OBS_DOMAINd01、OBS_DOMAINd02）。

## 四、WRF模式中使用Nudging


![Image Name](https://cdn.kesci.com/upload/image/rgahc83qzu.png?imageView2/0/w/960/h/960)


OBSGRID模块最终生成metoa_em.d0*、OBS_DOMIANd0*、wrfsfdda_d0*文件，其作用如下：1）metoa_em.d0*文件为后续Nudging提供了初始和边界条件，它与met_em.d0*文件作用类似，只不过metoa_em.d0*文件中融合了观测资料，将metoa_em.d0*文件链接至WRF模式em_real文件夹下并重命名为met_em.d0*，运行./real.exe会生成wrffdda文件。2）wrfsfdda 文件是进行Surface Analysis Nudging必备的文件，主要提供地表的信息。3）OBS_DOMIANd0*文件是进行Observation Nudging必备的文件，主要提供观测站点信息。

WRF模式中进行Nudging时，需先将OSGRID文件生成的metoa_em.d0*、OBS_DOMIANd0*、wrfsfdda_d0*文件链接至em_real文件夹下，再在namelist.input文件中&fdda模块添加相应的Nuding设置，运行./real.exe和./wrf.exe即可。值得注意的是，若要进行Observation Nudging 必须将OBS_DOMIANd0*文件链接至em_real文件夹下，若是Analysis Nudging，不需要链接OBS_DOMIANd0*文件。若要进行 Surface Analysis Nudging，必须链接wrfsfdda 文件，通常建议Surface Analysis Nudging和3D Analysis Nudging一起使用，即将wrfsfdda和wrffdda文件都作为WRF模式的输入文件。此外，Observation Nudging和Analysis Nudging还可同用，如在粗网格区域进行Analysis Nudging，在细网格区域进行Observation Nudging，这样搭配使用效果较好。由于边界层以下存在较多湍流，再分析资料在这一层结精确度较低，通常在边界层内关闭温度和湿度场的同化，仅允许边界层高度以上进行温湿度的同化，这样模型产生的边界层结构由模式中的物理学主导，风场的同化通常在所有模式层进行，关于WRF模式中Nudging的设置可参考[4]~[6]文章。

与四维变分同化相比，Nudging 不需要计算伴随矩阵和背景误差协方差矩阵，计算成本较低，相对较为成熟且易于实现，可操作性强，但是Nudging只能同化模式变量。

## 参考

[1] [WRF User’s Guide Chapter 7: Objective Analysis (OBSGRID)](https://www2.mmm.ucar.edu/wrf/users/docs/user_guide_v4/v4.4/users_guide_chap7.html)

[2] [Observation Nudging同化教程](https://blog.csdn.net/weixin_45372694/category_10278097.html)

[3] [WRF OBSGRID的简介与安装](https://blog.csdn.net/weixin_42487488/article/details/116034050)

[4] Deng A , Stauffer D , Gaudet B , et al. 1.9 UPDATE ON WRF-ARW END-TO-END MULTI-SCALE FDDA SYSTEM. 2009.

[5] Lauvaux T , Miles N L , Richardson S J , et al. Urban Emissions of CO2 from Davos, Switzerland: The First Real-Time Monitoring System Using an Atmospheric Inversion Technique[J]. Journal of Applied Meteorology and Climatology, 2013, 52(12):2654-2668.

[6] Lauvaux T , Miles N L , Deng A , et al. High-resolution atmospheric inversion of urban CO_2 emissions during the dormant season of the Indianapolis Flux Experiment (INFLUX).